[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/agents/smolagents_hf_with_mongodb.ipynb)

# Using Smolagents with MongoDB Atlas

This notebook demonstrates how to use [Smolagents](https://github.com/huggingface/smolagents) to interact with MongoDB Atlas for building AI-powered applications. We'll explore how to create tools that leverage MongoDB's aggregation capabilities to analyze and extract insights from data.

## Prerequisites

Before running this notebook, you'll need:

1. A MongoDB Atlas account and cluster
2. Python environment with required packages
3. OpenAI API key for GPT-4 access

## Setting Up MongoDB Atlas

1. Create a free MongoDB Atlas account at [https://www.mongodb.com/cloud/atlas/register](https://www.mongodb.com/cloud/atlas/register)
2. Create a new cluster (free tier is sufficient)
3. Configure network access by adding your IP address
4. Create a database user with read/write permissions
5. Get your connection string from Atlas UI (Click "Connect" > "Connect your application")
6. Replace `<password>` in the connection string with your database user's password
7. Enable network access from your IP address in the Network Access settings

## Observations

In this notebook, we:
- Define tools that interact with MongoDB Atlas using pymongo
- Use aggregation pipelines to analyze data
- Sample documents to understand schema structure
- Demonstrate how LLMs can generate and execute MongoDB queries

The tools showcase how to:
1. Execute aggregation pipelines generated by the LLM
2. Sample documents to understand collection structure
3. Handle errors and provide meaningful feedback

### Security Considerations

When working with MongoDB Atlas:
- Never commit connection strings with credentials to version control
- Use environment variables or secure secret management
- Restrict database user permissions to only what's needed
- Enable IP allowlist in Atlas Network Access settings

In [1]:
pip install pymongo smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 

In [2]:
import getpass
import os

MONGODB_URI = getpass.getpass("Enter your MongoDB Atlas URI: ")
os.environ["MONGODB_URI"] = MONGODB_URI

Enter your MongoDB Atlas URI: ··········


## Loading the dataset

In this example I am using the airbnb data set from https://huggingface.co/datasets/MongoDB/airbnb_embeddings .

- Database : ai_airbnb
- Collection : rentals

## Defining the tools

We'll create two main tools for interacting with MongoDB:

1. **Aggregation Tool**: Executes aggregation pipelines generated by the LLM to analyze data
   - Takes a pipeline as input
   - Handles complex data transformations
   - Returns aggregated results

2. **Sampling Tool**: Helps understand collection structure
   - Randomly samples documents
   - Provides schema insights
   - Useful for data exploration

Both tools automatically exclude embedding fields to reduce response size and improve readability.

In [3]:
import getpass
import json
import os

from google.colab import userdata
from pymongo import MongoClient
from smolagents import LiteLLMModel, tool
from smolagents.agents import ToolCallingAgent

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Choose which LLM engine to use!  Using Gemini is not directly supported by smolagents.
# You would need to integrate with Gemini's API.  This example continues with gpt-4o.
model = LiteLLMModel(model_id="gpt-4o")

client = MongoClient(MONGODB_URI, appname="devrel.showcase.smolagents")


@tool
def get_aggregated_docs(pipeline: str) -> list:
    """
    Gets a generated pipeline as 'pipeline' by the LLM and provide the context documents

     Args:
        pipeline: An array List with the current stages from the LLM # Added (list) and a description after the argument name
    """
    db = client["ai_airbnb"]
    collection = db["rentals"]
    pipeline = json.loads(pipeline)
    pipeline.insert(
        0, {"$project": {"text_embeddings": 0, "image_embeddings": 0}}
    )  # Use insert to add at the beginning
    docs = list(collection.aggregate(pipeline))
    return docs


@tool
def sample_documents(collection_name: str) -> str:
    """
    Use $sample to sample the collection docs

    Args:
      collection_name: The name of the collection to sample from
    """
    db = client["ai_airbnb"]
    try:
        collection = db[collection_name]
        sample = list(
            collection.aggregate(
                [
                    {"$project": {"text_embeddings": 0, "image_embeddings": 0}},
                    {"$sample": {"size": 5}},
                ]
            )
        )  # Sample 5 documents
        return sample
    except Exception as e:
        return f"Error: {e}"


agent = ToolCallingAgent(tools=[get_aggregated_docs, sample_documents], model=model)

# Example usage
user_query = "What are the supported countries in our 'rentals' collection, sample for structre and then  aggregate how many are in each country"
response = agent.run(user_query)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ What are the supported countries in our 'rentals' collection, sample for structre and then  aggregate how many are in each country                                                                   │
│                                                                                                                                                                                                      │
╰─ LiteLLMModel - gpt-4o ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sample_documents' with arguments: {'collection_name': 'rentals'}                                                                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: [{'_id': 23251205, 'listing_url': 'https://www.airbnb.com/rooms/23251205', 'name': 'Kitnet entre a zona sul e o centro.', 'summary': 'Kitnet, mezanino transformado em quarto,  sala com 
tv, wifi, geladeira, mesa retrátil para refeições, cozinha com cooktop de quatro bocas, forno elétrico, pia c/ água quente, banheiro c/ máquina de lavar samsung, aquecedor, água quente na pia e 
chuveiro.  Ambiente claro, arejado, acochegante, seguro, portaria 24 hs, 4 elevadores.', 'space': 'Espaço ótimo para quem deseja conhecer o melhor do Rio de Janeiro.', 'description': 'Kitnet, mezanino
transformado em quarto,  sala com tv, wifi, geladeira, mesa retrátil para refeições, cozinha com cooktop de quatro bocas, forno elétrico, pia c/ água quente, banheiro c/ máquina de lavar samsung, 
aquecedor, água quente na pia e chuveiro.  Ambiente claro, arejado, acochegante, seguro, portaria 24 hs, 4 elevadores. Espaço ótimo para quem deseja conhecer o melhor do Rio de Janeiro. Ambiente ideal
para um casal, porem acomoda bem crianças Localização privilegiada, Zona sul, Centro Rio de Janeiro e Santa Teresa do próximo ao maior centro de intreterimento do Rio (Lapa), casa de show, arco da 
lapa, a 5 min. do Metrô Glória, Praia do Flamengo, Aterro do Flamengo, Kitnet planejada com vista mar, aterro do Flamengo, Pça Paris. Ônibus, taxi, urber, principalmente metrô.', 
'neighborhood_overview': 'Localização privilegiada, Zona sul, Centro Rio de Janeiro e Santa Teresa do próximo ao maior centro de intreterimento do Rio (Lapa), casa de show, arco da lapa, a 5 min. do 
Metrô Glória, Praia do Flamengo, Aterro do Flamengo, Kitnet planejada com vista mar, aterro do Flamengo, Pça Paris.', 'notes': '', 'transit': 'Ônibus, taxi, urber, principalmente metrô.', 'access': 
'Ambiente ideal para um casal, porem acomoda bem crianças', 'interaction': '', 'house_rules': '- Horário de silêncio 22 hs', 'property_type': 'Loft', 'room_type': 'Entire home/apt', 'bed_type': 'Real 
Bed', 'minimum_nights': 5, 'maximum_nights': 30, 'cancellation_policy': 'flexible', 'last_scraped': datetime.datetime(2019, 2, 11, 5, 0), 'calendar_last_scraped': datetime.datetime(2019, 2, 11, 5, 0),
'first_review': None, 'last_review': None, 'accommodates': 2, 'bedrooms': 0.0, 'beds': 1.0, 'number_of_reviews': 0, 'bathrooms': 1.0, 'amenities': ['TV', 'Wifi', 'Air conditioning', 'Kitchen', 
'Elevator', 'Essentials', 'Iron'], 'price': 149, 'security_deposit': 500.0, 'cleaning_fee': 150.0, 'extra_people': 50, 'guests_included': 1, 'images': {'thumbnail_url': '', 'medium_url': '', 
'picture_url': 'https://a0.muscache.com/im/pictures/1a6e48f7-b065-41a5-8494-5f373b8b18b0.jpg?aki_policy=large', 'xl_picture_url': ''}, 'host': {'host_id': '107565373', 'host_url': 
'https://www.airbnb.com/users/show/107565373', 'host_name': 'Lázaro', 'host_location': 'BR', 'host_about': '', 'host_response_time': None, 'host_thumbnail_url': 
'https://a0.muscache.com/im/pictures/user/e6fbe872-ef0c-4708-b0d5-af2f645c5585.jpg?aki_policy=profile_small', 'host_picture_url': 
'https://a0.muscache.com/im/pictures/user/e6fbe872-ef0c-4708-b0d5-af2f645c5585.jpg?aki_policy=profile_x_medium', 'host_neighbourhood': 'Santa Teresa', 'host_response_rate': None, 'host_is_superhost': 
False, 'host_has_profile_pic': True, 'host_identity_verified': False, 'host_listings_count': 1, 'host_total_listings_count': 1, 'host_verifications': ['email', 'phone']}, 'address': {'street': 
'Centro, Rio de Janeiro, Brazil', 'suburb': 'Santa Teresa', 'government_area': 'Santa Teresa', 'market': 'Rio De Janeiro', 'country': 'Brazil', 'country_code': 'BR', 'location': {'type': 'Point', 
'coordinates': [-43.1775829067, -22.9182368387], 'is_location_exact': True}}, 'availability': {'availability_30': 0, 'availability_60': 0, 'availability_90': 0, 'availability_365': 0}, 
'review_scores': {'review_scores_accuracy': None, 'review_scores_cleanliness': None, 'review_scores_checkin': None, 'review_scores_communication': None, 'review_scores_location': None, 
're

...

[Step 2: Duration 1.78 seconds| Input tokens: 35,013 | Output tokens: 153]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "The supported countries in the 'rentals' collection and the number of listings in each are as follows:\n\n- Portugal: 555 listings\n-       │
│ Spain: 633 listings\n- Brazil: 606 listings\n- Hong Kong: 600 listings\n- Australia: 610 listings\n- China: 19 listings\n- Canada: 649 listings\n- United States: 1222 listings\n- Turkey: 661       │
│ listings"}                                                                                                                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The supported countries in the 'rentals' collection and the number of listings in each are as follows:

- Portugal: 555 listings
- Spain: 633 listings
- Brazil: 606 listings
- Hong Kong: 600 listings
- Australia: 610 listings
- China: 19 listings
- Canada: 649 listings
- United States: 1222 listings
- Turkey: 661 listings

[Step 3: Duration 7.77 seconds| Input tokens: 52,309 | Output tokens: 254]

## Vector Search based RAG with Atlas Search

Vector search allows us to find relevant documents based on the semantic meaning of the query rather than just keyword matching. In this section, we demonstrate how to build a Retrieval-Augmented Generation (RAG) agent that leverages MongoDB Atlas Search's vector search capabilities.

The RAG agent uses the `vector_search_rentals` tool to find relevant documents based on the query's embeddings. This approach enhances the search results by considering the context and meaning of the query, providing more accurate and relevant results.

We define the `vector_search_rentals` tool to perform the vector search and integrate it with the `ToolCallingAgent` to handle user queries effectively. The agent processes the query, performs the vector search, and returns the most relevant documents from the rentals collection.

### Create the vector search index if it does not exists

To create the vector search index, we define a search index model with the necessary configuration for vector search. This includes specifying the number of dimensions and the similarity metric. The index is then created on the text_embeddings field of the rentals collection. We also include a polling mechanism to ensure the index is ready for querying before proceeding.


In [ ]:
import json
import time

from pymongo.operations import SearchIndexModel

db = client["ai_airbnb"]
collection = db["rentals"]


## create index
search_index_model = SearchIndexModel(
    definition={
        "fields": [
            {
                "type": "vector",
                "numDimensions": 1536,
                "path": "text_embeddings",
                "similarity": "cosine",
            },
        ]
    },
    name="vector_index",
    type="vectorSearch",
)
result = collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")


def check_queryable(index):
    """Check if the index is queryable."""
    return index.get("queryable") is True


predicate = None
if predicate is None:
    predicate = check_queryable
while True:
    indices = list(collection.list_search_indexes(result))
    if len(indices) and predicate(indices[0]):
        break
    time.sleep(5)

print(result + " is ready for querying.")
client.close()

In [15]:
import json
import os

from litellm import embedding
from pymongo import MongoClient

# Assuming MONGODB_URI and OPENAI_API_KEY are already set as in the original code


@tool
def vector_search_rentals(query: str) -> list:
    """
    Gets a query , generates embeddings and locate vector store relavant documents

    Args:
      query: The query to search for

    Returns:
      A list of documents that are relavant to the query

    """
    response = embedding(model="text-embedding-3-small", input=[query])
    query_embedding = response["data"][0]["embedding"]

    # Perform vector search using Atlas Search
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "text_embeddings",
                "numCandidates": 100,
                "limit": 5,
            }
        },
        {
            "$project": {
                "text_embeddings": 0,
                "image_embeddings": 0,
                "_id": 0,
                "score": {"$meta": "searchScore"},
            }
        },
    ]

    results = list(collection.aggregate(pipeline))
    return results


# Example usage
user_query: str = "Show me apartments in London"
search_results = vector_search_rentals(user_query)

print(search_results)

[-0.03243120759725571, -0.006404194515198469, -0.03721725940704346, 0.04150191694498062, -0.04900006577372551, -0.03714888542890549, -0.03760470077395439, -0.021183982491493225, 0.005068088416010141, 0.007555126212537289, -0.0011138968402519822, -0.02404421754181385, 0.028100967407226562, 0.02173095941543579, 0.0163409523665905, -0.01394792553037405, -0.019622817635536194, -0.008677570149302483, 0.015623044222593307, 0.07151730358600616, 0.011703038588166237, -0.018927698954939842, -0.004549599252641201, -0.011748620308935642, -0.025252126157283783, -0.03382144123315811, -0.00747535889968276, -0.007765940856188536, 0.037855397909879684, 0.028374455869197845, -0.01858583837747574, -0.019862119108438492, -0.0035496558994054794, 0.036100514233112335, -0.002606689464300871, -0.012147458270192146, -0.014506298117339611, 0.013685832731425762, -0.020443283021450043, 0.00987408310174942, 0.02869352698326111, 0.04049912467598915, -5.906893784413114e-05, -0.0373540036380291, 0.010865479707717896

In [22]:
# prompt: Lets build a RAG smolagent that uses the vector store as context for queries

import json
import os

from pymongo import MongoClient
from smolagents import tool
from smolagents.agents import ToolCallingAgent

user_query = "Near parks and in brooklyn"

rag_agent = ToolCallingAgent(tools=[vector_search_rentals], model=model)

response = rag_agent.run(user_query)  # Pass context to agent.run()

╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ Near parks and in brooklyn                                                                                                                                                                           │
│                                                                                                                                                                                                      │
╰─ LiteLLMModel - gpt-4o ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'vector_search_rentals' with arguments: {'query': 'near parks in Brooklyn'}                                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[-0.03366561606526375, -0.023770568892359734, 0.004819623194634914, 0.007212277967482805, -0.014087649993598461, -0.03072080947458744, -0.003167849499732256, -0.017955826595425606, 0.0021056607365608215, 0.01166068110615015, 0.04090284928679466, 0.01356357429176569, 0.042150646448135376, -0.012428076937794685, 0.020139474421739578, 0.006432403344660997, 0.010874567553400993, -0.027751049026846886, 0.05959487706422806, -0.00046753467177040875, 0.04364800825715065, 0.017406795173883438, -0.01572226732969284, -0.026153866201639175, -0.0062514725141227245, 0.008934239856898785, 0.01026314590126276, 0.005580780562013388, 0.04010425880551338, 0.013289058580994606, -0.041876133531332016, -0.0196528322994709, -0.01900397799909115, -0.0009701636736281216, 0.0037496357690542936, -0.016346165910363197, -0.010550139471888542, -0.047341492027044296, 0.01292719691991806, 0.03918088600039482, 0.07501766830682755, 0.026553161442279816, -0.023046845570206642, -0.03825751692056656, -0.01153590064495802,

Observations: [{'listing_url': 'https://www.airbnb.com/rooms/223930', 'name': 'Lovely Apartment', 'summary': '', 'space': 'Travel to an amazing part of Brooklyn- Here you will find the Brooklyn 
Museum, Prospect Park, the Botanical Gardens and a slew of restaurants that will satisfy any palette. All less than a 5min walk from the apartment.  Subway lines are close by- within a 5 -10 minute 
walk to the 2, 3, Q, B, A, C.  The apartment is cozy and warm. It is great for couples or families. The unit is equip with Wi-Fi, Cable, TV and a full Kitchen.', 'description': 'Travel to an amazing 
part of Brooklyn- Here you will find the Brooklyn Museum, Prospect Park, the Botanical Gardens and a slew of restaurants that will satisfy any palette. All less than a 5min walk from the apartment.  
Subway lines are close by- within a 5 -10 minute walk to the 2, 3, Q, B, A, C.  The apartment is cozy and warm. It is great for couples or families. The unit is equip with Wi-Fi, Cable, TV and a full 
Kitchen.', 'neighborhood_overview': '', 'notes': '', 'transit': '', 'access': '', 'interaction': '', 'house_rules': '', 'property_type': 'Apartment', 'room_type': 'Entire home/apt', 'bed_type': 'Real 
Bed', 'minimum_nights': 5, 'maximum_nights': 60, 'cancellation_policy': 'moderate', 'last_scraped': datetime.datetime(2019, 3, 7, 5, 0), 'calendar_last_scraped': datetime.datetime(2019, 3, 7, 5, 0), 
'first_review': datetime.datetime(2011, 9, 23, 4, 0), 'last_review': datetime.datetime(2018, 9, 18, 4, 0), 'accommodates': 2, 'bedrooms': 1.0, 'beds': 1.0, 'number_of_reviews': 19, 'bathrooms': 1.0, 
'amenities': ['TV', 'Cable TV', 'Wifi', 'Air conditioning', 'Kitchen', 'Heating', 'Family/kid friendly', 'Smoke detector', 'Carbon monoxide detector', 'Fire extinguisher', 'Essentials', 'Shampoo', 
'Hangers', 'Iron', 'Laptop friendly workspace', 'Private living room', 'Hot water', 'Bed linens', 'Extra pillows and blankets', 'Ethernet connection', 'Microwave', 'Coffee maker', 'Refrigerator', 
'Dishes and silverware', 'Cooking basics', 'Oven', 'Stove', 'Long term stays allowed', 'Wide hallway clearance', 'Step-free access', 'Wide doorway', 'Wide clearance to bed', 'Accessible-height bed', 
'Step-free access', 'Wide doorway', 'Accessible-height toilet', 'Step-free access', 'Wide entryway', 'Handheld shower head'], 'price': 150, 'security_deposit': None, 'cleaning_fee': 100.0, 
'extra_people': 0, 'guests_included': 1, 'images': {'thumbnail_url': '', 'medium_url': '', 'picture_url': 'https://a0.muscache.com/im/pictures/2027724/4ea9761d_original.jpg?aki_policy=large', 
'xl_picture_url': ''}, 'host': {'host_id': '1164642', 'host_url': 'https://www.airbnb.com/users/show/1164642', 'host_name': 'Rosalynn', 'host_location': 'Brooklyn', 'host_about': 'I am a costumer in 
theater, tv/film.', 'host_response_time': 'within a day', 'host_thumbnail_url': 'https://a0.muscache.com/im/users/1164642/profile_pic/1316557315/original.jpg?aki_policy=profile_small', 
'host_picture_url': 'https://a0.muscache.com/im/users/1164642/profile_pic/1316557315/original.jpg?aki_policy=profile_x_medium', 'host_neighbourhood': 'Prospect Heights', 'host_response_rate': 50, 
'host_is_superhost': False, 'host_has_profile_pic': True, 'host_identity_verified': False, 'host_listings_count': 1, 'host_total_listings_count': 1, 'host_verifications': ['email', 'phone', 
'reviews']}, 'address': {'street': 'Brooklyn, NY, United States', 'suburb': 'Brooklyn', 'government_area': 'Prospect Heights', 'market': 'New York', 'country': 'United States', 'country_code': 'US', 
'location': {'type': 'Point', 'coordinates': [-73.96665, 40.67424], 'is_location_exact': True}}, 'availability': {'availability_30': 14, 'availability_60': 44, 'availability_90': 74, 
'availability_365': 349}, 'review_scores': {'review_scores_accuracy': 10, 'review_scores_cleanliness': 10, 'review_scores_checkin': 10, 'review_scores_communication': 10, 'review_scores_location': 10,
'review_scores_value': 9, 'review_scores_rating': 96}, 'reviews': [

[Step 0: Duration 1.67 seconds| Input tokens: 987 | Output tokens: 20]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "Here are some rental options near parks in Brooklyn:\n\n1. [Lovely Apartment](https://www.airbnb.com/rooms/223930): This cozy apartment is  │
│ located in Prospect Heights, Brooklyn. It's less than a 5-minute walk from attractions like the Brooklyn Museum, Prospect Park, and the Botanical Gardens. The host describes the apartment as cozy  │
│ and warm, suitable for couples or families, with amenities like Wi-Fi, Cable, TV, and a full kitchen.\n\n2. [Room in just-refurbished, classic brownstone                                            │
│ flat.](https://www.airbnb.com/rooms/18194415): Located in Park Slope, Brooklyn, this private room in a classic brownstone provides a true New York local experience. The area is known for its       │
│ vibrant music scene and is close to Prospect Park. This rental is perfect for those who enjoy walking and exploring.\n\n3. [Wow Historical Brooklyn New                                              │
│ York!@!](https://www.airbnb.com/rooms/6146081): This entire apartment is located in East Flatbush, Brooklyn. It's a cozy two-bedroom apartment with a homely feel and is only a short distance from  │
│ attractions like the Brooklyn Botanical Garden and Prospect Park.\n\n4. [Prime location: abundant stores & transportation!](https://www.airbnb.com/rooms/21871576): Situated in Flatlands, Brooklyn, │
│ this townhouse is located in a vibrant and peaceful neighborhood with convenient access to stores and public transportation, making it easy to explore Brooklyn and beyond.\n\n5. [Room in Prospect  │
│ Heights](https://www.airbnb.com/rooms/6171211): This private room in a shared apartment is right at Prospect Park and the Brooklyn Botanic Garden, offering a fantastic location for nature lovers.  │
│ The space includes a private porch, full kitchen, and laundry facilities.\n\nThese options provide a variety of accommodations, from entire homes to private rooms, each offering unique experiences │
│ and proximity to Brooklyn's parks and attractions. Prices and amenities vary, so you might want to consider your specific needs and preferences when choosing."}                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Here are some rental options near parks in Brooklyn:

1. [Lovely Apartment](https://www.airbnb.com/rooms/223930): This cozy apartment is located in Prospect Heights, Brooklyn. It's less than a 5-minute walk from attractions like the Brooklyn Museum, 
Prospect Park, and the Botanical Gardens. The host describes the apartment as cozy and warm, suitable for couples or families, with amenities like Wi-Fi, Cable, TV, and a full kitchen.

2. [Room in just-refurbished, classic brownstone flat.](https://www.airbnb.com/rooms/18194415): Located in Park Slope, Brooklyn, this private room in a classic brownstone provides a true New York 
local experience. The area is known for its vibrant music scene and is close to Prospect Park. This rental is perfect for those who enjoy walking and exploring.

3. [Wow Historical Brooklyn New York!@!](https://www.airbnb.com/rooms/6146081): This entire apartment is located in East Flatbush, Brooklyn. It's a cozy two-bedroom apartment with a homely feel and is
only a short distance from attractions like the Brooklyn Botanical Garden and Prospect Park.

4. [Prime location: abundant stores & transportation!](https://www.airbnb.com/rooms/21871576): Situated in Flatlands, Brooklyn, this townhouse is located in a vibrant and peaceful neighborhood with 
convenient access to stores and public transportation, making it easy to explore Brooklyn and beyond.

5. [Room in Prospect Heights](https://www.airbnb.com/rooms/6171211): This private room in a shared apartment is right at Prospect Park and the Brooklyn Botanic Garden, offering a fantastic location 
for nature lovers. The space includes a private porch, full kitchen, and laundry facilities.

These options provide a variety of accommodations, from entire homes to private rooms, each offering unique experiences and proximity to Brooklyn's parks and attractions. Prices and amenities vary, so
you might want to consider your specific needs and preferences when choosing.

[Step 1: Duration 8.35 seconds| Input tokens: 22,812 | Output tokens: 454]



## Conclusions

This notebook successfully demonstrates the integration of Smolagents with MongoDB Atlas, enabling effective data analysis through an AI agent.  The defined tools, `get_aggregated_docs` and `sample_documents`, effectively interact with the Airbnb dataset stored in MongoDB Atlas.  The agent, powered by a chosen LLM (in this case, GPT-4o), successfully translates user queries into both data sampling and aggregation pipelines executed against the MongoDB database.

Key improvements and observations include:

* **Robust Tool Design:** The tools now incorporate error handling, providing more informative feedback to the user in case of issues.  The exclusion of embedding fields from queries enhances performance and readability of results.
* **Enhanced Query Handling:**  The inclusion of an initial projection stage in the aggregation pipeline, specifically designed to remove embedding fields (`text_embeddings` and `image_embeddings`) prior to other stages, ensures more efficient query execution and smaller response sizes.  The use of `json.loads()` ensures that the pipeline string received from the LLM is correctly parsed.
Atlas Search excels at finding relevant documents quickly, thanks to its vector search capabilities.  This is particularly beneficial for large datasets where traditional keyword search may be insufficient.
* **Improved User Experience:** Clearer tool documentation and example usage further enhance the user's ability to interact with the agent and interpret results.
* **Practical Application:** The demonstration showcases a practical application for analyzing data within a MongoDB Atlas database using an LLM-powered agent.

Future development could explore:

* **Expanded Toolset:** Implementing additional tools for data manipulation, filtering, and more complex analytics.
* **Advanced Query Generation:** Exploring methods to refine the LLM's ability to generate accurate and efficient MongoDB queries.
* **Visualization Capabilities:** Integrating data visualization libraries to present the analysis results more effectively.
* **Security Enhancements:** Further solidifying security practices, potentially incorporating environment variable management for sensitive credentials.